# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [2]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Prostate Cancer"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [5]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Prostate_Cancer_(PRAD)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.PRAD.sampleMap_PRAD_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.PRAD.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode',
 'bcr_patient_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'biochemical_recurrence', 'bone_scan_results', 'clinical_M', 'clinical_T', 'days_to_birth', 'days_to_bone_scan_performed', 'days_to_collection', 'days_to_death', 'days_to_diagnostic_computed_tomography_performed', 'days_to_d

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_diagnostic_computed_tomography_performed', 'days_to_diagnostic_mri_performed', 'days_to_first_biochemical_recurrence', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_psa', 'days_to_second_biochemical_recurrence', 'days_to_third_biochemical_recurrence', 'form_completion_date', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [51, 57, 47, 52, 70],
 'days_to_birth': [-18658.0, -20958.0, -17365.0, -19065.0, -25904.0],
 'days_to_collection': [1273.0, 1290.0, 923.0, 1155.0, 922.0],
 'days_to_death': [nan, nan, nan, nan, nan],
 'days_to_diagnostic_computed_tomography_performed': [nan,
  11.0,
  -2.0,
  29.0,
  32.0],
 'days_to_diagnostic_mri_performed': [nan, 84.0, 4.0, 204.0, 89.0],
 'days_to_first_biochemical_recurrence': [nan, nan, nan, nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [621.0, 1701.0, 1373.0, 671.0, 1378.0],
 'days_to_new_tumor_event_after_initial_treatment': [nan, nan, nan, nan, nan],
 'days_to_psa': ['585', '1332', '995', '560', '1033'],
 'days_to_second_biochemical_recurrence': [nan, nan, nan, nan, nan],
 'days_to_third_biochemical_recurrence': [nan, nan, nan, nan, nan],
 'form_completion_date': ['2015-3-31',
  '2015-3-31',
  '2015-3-31',
  '2015-3-31',
  '2015-3-31'],
 'year_of_initial_pathologi

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Prostate Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-2A-A8VL-01,1,51,1,-0.529292,1.057174,-0.531035,0.618328,0.190622,0.81449,0.190406,...,-0.722986,0.512518,-0.149678,-0.565717,-1.980333,-1.28139,-0.440177,-1.480327,-0.571195,1.802067
TCGA-2A-A8VO-01,1,57,1,-0.758992,1.607474,-0.531035,0.457328,-0.006778,0.08739,0.116506,...,-0.842386,0.648518,1.122422,0.731283,-1.019233,-1.28139,-0.325877,-0.336327,-0.120095,2.618067
TCGA-2A-A8VT-01,1,47,1,-0.205792,-0.895626,-0.531035,0.330028,0.326922,0.00119,0.008406,...,-0.102786,-0.086682,-0.748878,-0.282217,-1.477333,-1.28139,-1.004377,-0.619527,-0.764495,1.723367
TCGA-2A-A8VV-01,1,52,1,-0.697292,1.133574,-0.531035,0.807328,-0.273778,0.17979,0.243206,...,-0.554086,0.469718,-0.748878,-0.630117,-1.643033,-1.28139,-0.389677,-1.920727,-0.447995,2.168967
TCGA-2A-A8VX-01,1,70,1,-0.847392,1.244074,-0.531035,0.677428,0.161422,0.57909,0.142406,...,-1.060086,-0.086682,0.579622,-0.998517,-2.466333,-1.28139,-0.421977,-3.027827,-1.182695,-0.682033


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 550 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Prostate Cancer', the least common label is '0' with 52 occurrences. This represents 9.45% of the dataset.
The distribution of the feature 'Prostate Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 56.0
  50% (Median): 61.0
  75%: 66.0
Min: 41
Max: 78
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 550 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: /content/drive/MyDrive/AI4Science_Public/output1/Shuyi/Prostate-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "Underweight"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE84954',
 'GSE57802',
 'GSE85017',
 'GSE163901',
 'GSE163902',
 'GSE50982',
 'GSE163903',
 'GSE119483',
 'GSE130563',
 'GSE131835',
 'GSE10135']

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE130563"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/content/drive/MyDrive/AI4Science_Public/data/GEO/Underweight/GSE130563/GSE130563_family.soft.gz',
 '/content/drive/MyDrive/AI4Science_Public/data/GEO/Underweight/GSE130563/GSE130563_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Skeletal Muscle Fibrosis in Pancreatic Cancer Patients with Respect to Survival"
!Series_summary	"Skeletal muscle wasting is a devastating consequence of cancer that may be responsible for nearly 30% of cancer-related deaths. In addition to muscle atrophy, we have identified significant muscle fiber damage and replacement of muscle with fibrotic tissue in rectus abdominis muscle biopsies from cachectic pancreatic ductal adenocarcinoma (PDAC) patients that associates with poor survival. Transcriptional profiling of muscle harvested from these same patients supported these findings by identifying gene clusters related to wounding, inflammation and cellular response to TGF-B upregulated in cachectic PDAC patients compared with non-cancer controls."
!Series_summary	"In this dataset, we include the expression data obtained from rectus abdominis muscle biopsies fron non-cancer controls patients undergoing abdominal surgery for benign reasons and from PDAC patients undergoing t

In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM3743555,GSM3743556,GSM3743557,GSM3743558,GSM3743559,GSM3743560,GSM3743561,GSM3743562,GSM3743563,...,GSM3743591,GSM3743592,GSM3743593,GSM3743594,GSM3743595,GSM3743596,GSM3743597,GSM3743598,GSM3743599,GSM3743600
0,!Sample_characteristics_ch1,diagnosis: mucinous cystadenoma,diagnosis: squamoid cyst,diagnosis: IPMN,diagnosis: bile duct injury,diagnosis: choledocholithiasis s/p cholecystec...,diagnosis: common bile duct stricture,diagnosis: cholecystitis,diagnosis: bile duct injury and acute pancreat...,diagnosis: open cholecystectomy,...,diagnosis: pancreatic ductal adenocarcinoma,diagnosis: pancreatic ductal adenocarcinoma,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis
1,!Sample_characteristics_ch1,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,...,Sex: F,Sex: M,Sex: M,Sex: M,Sex: M,Sex: F,Sex: F,Sex: F,Sex: M,Sex: M
2,!Sample_characteristics_ch1,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,...,tnm: pT3N1M0,tnm: pT3N0M0,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a
3,!Sample_characteristics_ch1,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 4,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 3,...,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...
4,!Sample_characteristics_ch1,age: 33,age: 68,age: 73,age: 49,age: 78,age: 57,age: 55,age: 50,age: 47,...,age: 59,age: 74,age: 30,age: 51,age: 55,age: 55,age: 45,age: 58,age: 50,age: 54


In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['diagnosis: mucinous cystadenoma',
  'diagnosis: squamoid cyst',
  'diagnosis: IPMN',
  'diagnosis: bile duct injury',
  'diagnosis: choledocholithiasis s/p cholecystectomy',
  'diagnosis: common bile duct stricture',
  'diagnosis: cholecystitis',
  'diagnosis: bile duct injury and acute pancreatic necrosis',
  'diagnosis: open cholecystectomy',
  'diagnosis: acute pancreatic necrosis',
  'diagnosis: sclerosing cholangitis',
  'diagnosis: stricture of choledochojejunostomy',
  'diagnosis: common bile duct injury',
  'diagnosis: choledochal cyst',
  'diagnosis: pancreatic ductal adenocarcinoma',
  'diagnosis: chronic pancreatitis'],
 1: ['Sex: F', 'Sex: M'],
 2: ['tnm: n/a',
  'tnm: pT3N1M0',
  'tnm: pT3N0M0',
  'tnm: Stage IV',
  'tnm: pT4N1M0',
  'tnm: pT1N0M0',
  'tnm: pT2N1M0'],
 3: ['bw loss in 6 months prior to surgery: 0',
  'bw loss in 6 months prior to surgery: 4',
  'bw loss in 6 months prior to surgery: 3',
  'bw loss in 6 months prior to surgery: 6.5',
  'bw loss in 6 m

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Underweight\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Underweight\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an in

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = True
trait_row_id = 3
age_row_id = 4
gender_row_id = 1

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    try:
        # Try converting the value to float
        return float(value)
    except (ValueError, TypeError):
        # Return None if conversion is not possible
        return None

def convert_age(value):
    try:
        # Try converting the value to int
        return int(value)
    except (ValueError, TypeError):
        # Return None if conversion is not possible
        return None

def convert_gender(value):
    if value.lower() == 'f':
        return 0
    elif value.lower() == 'm':
        return 1
    else:
        return None


In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM3743555,GSM3743556,GSM3743557,GSM3743558,GSM3743559,GSM3743560,GSM3743561,GSM3743562,GSM3743563,GSM3743564,...,GSM3743591,GSM3743592,GSM3743593,GSM3743594,GSM3743595,GSM3743596,GSM3743597,GSM3743598,GSM3743599,GSM3743600
Underweight,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Age,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Gender,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM3743555,GSM3743556,GSM3743557,GSM3743558,GSM3743559,GSM3743560,GSM3743561,GSM3743562,GSM3743563,...,GSM3743591,GSM3743592,GSM3743593,GSM3743594,GSM3743595,GSM3743596,GSM3743597,GSM3743598,GSM3743599,GSM3743600
0,100009613_at,2.484554,2.170547,2.240293,2.017631,2.322433,2.320112,2.274520,1.976751,2.118517,...,1.998159,2.574119,2.171907,2.048359,2.141913,2.141388,2.387593,1.884169,1.866739,2.190988
1,100009676_at,3.746875,3.229653,3.505413,3.202440,3.726104,2.772018,3.349980,3.028347,3.041839,...,3.200839,2.428775,3.050473,2.940380,3.182597,2.983551,3.162175,3.118802,2.762642,3.217956
2,10000_at,4.383380,4.920306,4.963512,4.992320,5.066081,4.510955,5.113312,4.481963,4.806586,...,4.880312,5.219858,5.391569,5.379981,5.002947,4.674068,4.673622,5.638959,4.856591,5.245355
3,10001_at,5.822047,5.975492,5.852250,6.243071,6.405402,6.347597,6.168092,6.317435,5.907259,...,6.260879,5.486075,6.186902,6.096248,5.928020,6.029681,6.109818,6.056322,6.086313,5.975695
4,10002_at,1.883013,2.279264,2.570026,2.095891,1.950774,1.904020,2.093506,2.002870,2.585459,...,2.015685,2.353116,1.956116,2.378803,1.884763,1.839770,2.079452,1.902106,1.917236,2.439328


In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['100009613_at',
 '100009676_at',
 '10000_at',
 '10001_at',
 '10002_at',
 '100033413_at',
 '100033422_at',
 '100033423_at',
 '100033424_at',
 '100033425_at',
 '100033426_at',
 '100033427_at',
 '100033428_at',
 '100033430_at',
 '100033431_at',
 '100033432_at',
 '100033434_at',
 '100033435_at',
 '100033436_at',
 '100033437_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at', '100033413_at', '100033422_at', '100033423_at', '100033424_at', '100033425_at', '100033426_at', '100033427_at', '100033428_at', '100033430_at', '100033431_at', '100033432_at', '100033434_at', '100033435_at', '100033436_at', '100033437_at']\n"

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    '''

In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 38 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Breast Cancer', the least common label is '1.0' with 19 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Breast Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 47.25
  50% (Median): 55.0
  75%: 62.25
Min: 37.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 11 occurrences. This represents 28.95% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [6]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'Xena'

In [7]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,550,


In [8]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [9]:
merged_data.head()

,Prostate Cancer,Age,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,51,-0.529292,1.057174,-0.531035,0.618328,0.190622,0.81449,0.190406,0.636354,...,-0.722986,0.512518,-0.149678,-0.565717,-1.980333,-1.28139,-0.440177,-1.480327,-0.571195,1.802067
1,1,57,-0.758992,1.607474,-0.531035,0.457328,-0.006778,0.08739,0.116506,0.655854,...,-0.842386,0.648518,1.122422,0.731283,-1.019233,-1.28139,-0.325877,-0.336327,-0.120095,2.618067
2,1,47,-0.205792,-0.895626,-0.531035,0.330028,0.326922,0.00119,0.008406,-0.469046,...,-0.102786,-0.086682,-0.748878,-0.282217,-1.477333,-1.28139,-1.004377,-0.619527,-0.764495,1.723367
3,1,52,-0.697292,1.133574,-0.531035,0.807328,-0.273778,0.17979,0.243206,-0.175046,...,-0.554086,0.469718,-0.748878,-0.630117,-1.643033,-1.28139,-0.389677,-1.920727,-0.447995,2.168967
4,1,70,-0.847392,1.244074,-0.531035,0.677428,0.161422,0.57909,0.142406,-1.165446,...,-1.060086,-0.086682,0.579622,-0.998517,-2.466333,-1.28139,-0.421977,-3.027827,-1.182695,-0.682033


In [10]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [11]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [12]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [13]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [14]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 51.82% ± 1.29%


In [15]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 75.09% ± 2.12%


In [16]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/usr/local/lib/python3.10/dist-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [17]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0092
Found 80 genes with non-zero coefficients associated with the trait 'Prostate Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
    UROC1    -1.478598              1.478598
   OR10A6    -1.161411              1.161411
  SULT1B1     1.031523              1.031523
  PRAMEF4     1.027987              1.027987
     HCN1    -1.006317              1.006317
 C17orf78     0.947944              0.947944
   TTTY1B     0.927874              0.927874
    TTTY8     0.917506              0.917506
  C4orf50     0.883372              0.883372
    KRT71     0.860023              0.860023
  NUP210L     0.755066              0.755066
     FTMT     0.741965              0.741965
     BEX4     0.698424              0.698424
LOC401127    -0.652678              0.652678
   TFAMP1     0.639533              0.639533
     S

/content/drive/MyDrive/Colab Notebooks/utils.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [18]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0092
Found 4 significant genes associated with the trait 'Prostate Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
    DLX1     0.100950 1.877746e-09           0.000033
   TDRD1     0.089702 1.390052e-07           0.001206
    ZIM3     0.048932 5.751709e-06           0.024944
 FAM74A1     0.039560 4.977430e-06           0.024944


/content/drive/MyDrive/Colab Notebooks/utils.py:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
